# 1.Import Dependencies

In [47]:
import os
import gymnasium
from pandas_ta import total_time
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Load Environment

In [48]:
environment_name = 'CartPole-v1'
env = gymnasium.make(environment_name, render_mode="human")

In [49]:
# episodes = 5
# for episode in range(1, episodes + 1):
#     state = env.reset()
#     done = False
#     score = 0
# 
#     while not done:
#         action = env.action_space.sample()
#         n_state, reward, terminated, truncated, info = env.step(action)
#         done = terminated or truncated
#         score += reward
# 
#     print(f'Episode:{episode}, Score:{score}')
# env.close()

# 3.Understanding the Environment

In [50]:
env.action_space

Discrete(2)

In [51]:
env.action_space.sample()

1

In [52]:
env.observation_space.sample()

array([ 3.5426612 ,  0.54937434, -0.26167846, -1.6856135 ], dtype=float32)

# 4. Train Model

In [53]:
log_path = os.path.join('Training', 'Logs')


In [54]:
import torch

# Check if MPS is available
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
    print("MPS is not available, using CPU.")



In [55]:
env = gymnasium.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path, device=device)

Using mps device


In [56]:
model.learn(total_timesteps=20000)

Logging to Training/Logs/PPO_3
-----------------------------
| time/              |      |
|    fps             | 401  |
|    iterations      | 1    |
|    time_elapsed    | 5    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 311         |
|    iterations           | 2           |
|    time_elapsed         | 13          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009432666 |
|    clip_fraction        | 0.115       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.685      |
|    explained_variance   | 0.000331    |
|    learning_rate        | 0.0003      |
|    loss                 | 8.01        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0192     |
|    value_loss           | 54.9        |
-----------------------------------------
---

# 5. Save Model & Load Model

In [57]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model_CartPole')

In [58]:
model.save(PPO_path)

In [59]:
del model

In [60]:
model = PPO.load(PPO_path, env=env)

# 6. Evaluate